In [1]:
!pip install audio2numpy
!pip install pydub


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Preparing metadata (setup.py) ... done
  Created wheel for ffmpeg: filename=ffmpeg-1.4-py3-none-any.whl size=6083 sha256=99d0a7e897fca8ede7f6dd9b1925b4fb7bc84edbb98e1afaaa6612ae666c245a
  Stored in directory: /root/.cache/pip/wheels/8e/7a/69/cd6aeb83b126a7f04cbe7c9d929028dc52a6e7d525ff56003a
Successfully built ffmpeg
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
# Connect Google Drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [43]:
import os
import shutil
import zipfile
from enum import Enum

import librosa as librosa
from dataclasses import dataclass
from enum import Enum
from typing import List, Optional

In [42]:
def get_duration(path):
    y, sr = librosa.load(path)
    return librosa.get_duration(y=y)

In [44]:
def delete_tree(path):
    if os.path.exists(path):
        if os.path.isdir(path):
            shutil.rmtree(path)
        else:
            os.remove(path)
    else:
        print("path doesn't exists")

In [49]:
def delete_audio_by_duration(files):
    for path in files:
        duration = get_duration(path)
        if duration < 15:
          print(f"duration : {duration} || path : {path}")
          delete_tree(path)

In [46]:
def filter_audio(dir_path):
    files = librosa.util.find_files(directory=dir_path, ext=['mp3', 'ogg'], recurse=True)
    delete_audio_by_duration(files)

In [ ]:
path_to_clean = "/content/drive/MyDrive"
filter_audio(path_to_clean)

In [47]:

class SectionName(Enum):
    General = "[General]"
    Editor = "[Editor]"
    Metadata = "[Metadata]"
    Difficulty = "[Difficulty]"
    Events = "[Events]"
    TimingPoints = "[TimingPoints]"
    Colours = "[Colours]"
    HitObjects = "[HitObjects]"


@dataclass
class Section:

    def value(self, value):
        if value.strip().isnumeric():
            return int(value)
        else:
            try:
                return float(value)
            except ValueError:
                return value

    def parse_line(self, line: str):
        ...


@dataclass
class HitSample:
    normalSet: int = 0  # SampleSet
    additionSet: int = 0  # SampleSet
    index: int = 0
    volume: int = 0
    filename: Optional[str] = None

    # def set(self, normalSet: int, additionSet: int, index: int, volume: int, filename: Optional[str] = ""):
    #     self.normalSet = normalSet
    #     self.additionSet = additionSet
    #     self.index = index
    #     self.volume = volume
    #     self.filename = filename

    def __str__(self):
        if self.filename is not None:
            return str(f"{self.normalSet}:{self.additionSet}:{self.index}:{self.volume}:{self.filename}:")
        else:
            return str(f"{self.normalSet}:{self.additionSet}:{self.index}:{self.volume}:")


@dataclass
class General(Section):
    AudioFilename: Optional[str] = None
    AudioLeadIn: int = 0
    AudioHash: Optional[str] = None  # deprecated
    PreviewTime: int = -1
    Countdown: int = 1
    SampleSet: str = "Normal"
    StackLeniency: float = 0.7
    Mode: int = 0
    LetterboxInBreaks: int = 0
    StoryFireInFront: int = 1  # deprecated
    UseSkinSprites: int = 0
    AlwaysShowPlayfield: int = 0  # deprecated
    OverlayPosition: str = "NoChange"
    SkinPreference: Optional[str] = None
    EpilepsyWarning: int = 0
    CountdownOffset: int = 0
    SpecialStyle: int = 0
    WidescreenStoryboard: int = 0
    SamplesMatchPlaybackRate: int = 0

    def parse_line(self, line: str):
        members = line.split(':')
        self.__setattr__(members[0], self.value(members[1]))


@dataclass
class Editor(Section):
    Bookmarks: Optional[List[int]] = None
    DistanceSpacing: float = 1.22  # between 0.1 and 2.0
    BeatDivisor: int = 4
    GridSize: int = 4
    TimelineZoom: float = 1.0  # between 0.1 and 8.0

    def parse_line(self, line: str):
        members = line.split(':')
        if members[0] == "Bookmarks":
            self.Bookmarks = [self.value(x) for x in members[1].split(",")]
        else:
            self.__setattr__(members[0], self.value(members[1]))


@dataclass
class Metadata(Section):
    Title: Optional[str] = None
    TitleUnicode: Optional[str] = None
    Artist: Optional[str] = None
    ArtistUnicode: Optional[str] = None
    Creator: Optional[str] = None
    Version: Optional[str] = None
    Source: Optional[str] = None
    Tags: Optional[List[str]] = None
    BeatmapID: int = 0
    BeatmapSetID: int = 0

    def parse_line(self, line: str):
        members = line.split(':')
        if members[0] == "Tags":
            self.Tags = [x for x in members[1].split(" ")]
        else:
            self.__setattr__(members[0], self.value(members[1]))


@dataclass
class Difficulty(Section):
    HPDrainRate: float = 5.0
    CircleSize: float = 5.0
    OverallDifficulty: float = 5.0
    ApproachRate: float = 5.0
    SliderMultiplier: float = 1.4
    SliderTickRate: float = 1.0

    def parse_line(self, line: str):
        members = line.split(':')
        self.__setattr__(members[0], self.value(members[1]))


class EventParams:
    pass


class Event(Section):
    eventType: str
    startTime: int
    eventParams: List[EventParams]


class Background(EventParams):
    filename: str
    xOffset: int
    yOffset: int


class Video(EventParams):
    Video: 1
    startTime: int
    filename: str
    xOffset: int
    yOffset: int


class Pause(EventParams):
    # 2:Break TODO check wiki because sintaxe is strange
    Break: 2
    startTime: int
    endTime: int


#  TODO
class Storyboard(EventParams):
    pass


@dataclass
class TimingPoint(Section):
    time: int = 0
    beatLength: float = 0
    meter: int = 0
    sampleSet: int = 1  # SampleSet = SampleSet.Normal.value
    sampleIndex: int = 0
    volume: int = 1
    uninherited: int = 0
    effects: int = 0  # Effect = None
    # bpm: Optional[int] = None

    def parse_line(self, line: str):
        members = line.split(",")
        self.time = self.value(members[0])
        self.beatLength = self.value(members[1])
        self.meter = self.value(members[2])
        self.sampleSet = self.value(members[3])
        self.sampleIndex = self.value(members[4])
        self.volume = self.value(members[5])
        self.uninherited = self.value(members[6])
        self.effects = self.value(members[7])
        # self.calculate_bpm()

    # def calculate_bpm(self):
    #     self.bpm = round(60000 / self.beatLength)

    def calculate_beat_length(self, bpm: int):
        self.beatLength = 60000 / bpm

    def __str__(self):
        return ",".join([str(value) for value in self.__dict__.values() if value != None])


class Colour:
    R: int
    G: int
    B: int


# TODO check wiki for colours
@dataclass
class ColourSection(Section):
    colours: List[Colour]
    slider_body: Colour
    slider_track_override: Colour
    slider_border: Colour

    # SliderTrackOverride
    # SliderBorder
    def parse_line(self, line):
        members = line.split(":")
        isCombo = line.startswith("Combo")
        split = members[1].split(",")
        if len(split) != 3 or len(split) != 4:
            print(" invalid color")
            return -1
        assert 0 <= split[0] <= 255
        assert 0 <= split[1] <= 255
        assert 0 <= split[2] <= 255

        if isCombo:
            # {"R": split[0], "G": split[1], "B": split[2]}
            self.colours.append(Colour(R=split[0], G=split[1], B=split[2]))
        else:
            # do nothing for the moment
            pass


@dataclass
class HitObject(Section):
    x: int = 0
    y: int = 0
    time: int = 0
    type: int = 0
    hitSound: int = 0
    hitSample: Optional[str] = HitSample().__str__()

    def __str__(self):
        return f"{self.x},{self.y},{self.time},{self.type},{self.hitSound},{self.hitSample}"

    def get_hit_sample(self, line) -> str:
        if self.has_hit_sample(line):
            return line
        return "0:0:0:0:0:"

    def has_hit_sample(self, line) -> bool:
        if type(line) == int or type(line) == float:
            return False
        else:
            return True


@dataclass
class Cercle(HitObject):

    def parse_line(self, line):
        members = line.split(",")
        self.x = self.value(members[0])
        self.y = self.value(members[1])
        self.time = self.value(members[2])
        self.type = self.value(members[3])
        self.hitSound = self.value(members[4])
        self.hitSample = self.get_hit_sample(self.value(members[-1]))


@dataclass
class Spinner(HitObject):
    endTime: int = 0

    def parse_line(self, line):
        members = line.split(",")
        self.x = self.value(members[0])
        self.y = self.value(members[1])
        self.time = self.value(members[2])
        self.type = self.value(members[3])
        self.hitSound = self.value(members[4])
        self.endTime = self.value(members[5])

        self.hitSample = self.get_hit_sample(self.value(members[-1]))


@dataclass
class CurvePoint:
    x: int = 0
    y: int = 0

    def __str__(self):
        return f"{self.x}:{self.y}"


@dataclass
class Slider(HitObject):
    curvePoints: List[CurvePoint] = None
    slides: int = 0
    length: float = 0.0
    edgeSounds: str = ""
    edgeSets: str = ""
    curveType: str = ""

    def parse_line(self, line):
        members = line.split(",")
        self.x = self.value(members[0])
        self.y = self.value(members[1])
        self.time = self.value(members[2])
        self.type = self.value(members[3])
        self.hitSound = self.value(members[4])

        # Parse slider points
        points = (members[5] or '').split('|')
        self.curveType = points[0]
        self.curvePoints = []
        if len(points):
            for i in range(1, len(points)):
                coordinates = points[i].split(':')
                curve_point = CurvePoint()
                curve_point.x = self.value(coordinates[0])
                curve_point.y = self.value(coordinates[1])
                self.curvePoints.append(curve_point)
                # self.curvePoints.append(curve_point.__str__())

        # Parse repeat slides bumber & length
        self.slides = int(members[6])
        self.length = int(round(float(members[7])))

        # Parse edgeSounds
        if len(members) > 9:
            if members[8]:
                self.edgeSounds = members[8]

            # Parse edgeSets
            if members[9]:
                self.edgeSets = members[9]

        self.hitSample = self.get_hit_sample(self.value(members[-1]))


In [5]:
import codecs
import os
import re
from typing import List

In [52]:
class Parse:
    def __init__(self):
        self.file_format = ""
        self.general = General()
        self.editor = Editor()
        self.metadata = Metadata()
        self.difficulty = Difficulty()
        self.events: List[Event] = []
        self.timing_points: List[TimingPoint] = []
        self.colours: List[ColourSection] = []
        self.hit_objects: List[HitObject] = []

        self.osu_section = ""

    def parse_hit_object_type(self, line):
        _type = int(line.split(",")[3].strip())
        # https://osu.ppy.sh/wiki/fr/Client/File_formats/Osu_%28file_format%29#type
        # convert in bit
        # 0: Cercle
        # 1: Slider
        # 3:Spinner
        # 7 osu mania
        if _type & 1:
            cercle = Cercle()
            cercle.parse_line(line)
            return cercle
        elif _type & 2:
            slider = Slider()
            slider.parse_line(line)
            return slider
        elif _type & 8:
            spinner = Spinner()
            spinner.parse_line(line)
            return spinner
        # elif _type & 128:
        #     print("mania")
        else:
            cercle = Cercle()
            cercle.parse_line(line)
            print("unknown type:", _type)
            return cercle

    def parse_line(self, line: str):
        line = line.strip()
        if not line:
            return

        match = re.search(r"\[(.*?)\]", line)
        if match:
            self.osu_section = match.group(0)
            return
        match = re.match('^osu file format (v[0-9]+)$', line)
        if match:
            # self.file_format = line
            self.file_format = match.group(1)
            return
        if self.osu_section == SectionName.General.value:
            self.general.parse_line(line)
        elif self.osu_section == SectionName.Editor.value:
            self.editor.parse_line(line)
        elif self.osu_section == SectionName.Metadata.value:
            self.metadata.parse_line(line)
        elif self.osu_section == SectionName.Difficulty.value:
            self.difficulty.parse_line(line)
        # elif self.osu_section == SectionName.Events.name:
        #     self.events_section.append(line)
        elif self.osu_section == SectionName.TimingPoints.value:
            timing_point = TimingPoint()
            timing_point.parse_line(line)
            self.timing_points.append(timing_point)
        # elif self.osu_section == SectionName.Colours.name:
        #     self.colours_section.append(line)
        elif self.osu_section == SectionName.HitObjects.value:
            hit_obj = self.parse_hit_object_type(line)
            self.hit_objects.append(hit_obj)

    def parse_file(self, file):
      if os.path.isfile(file):
          with codecs.open(file, 'r', encoding="utf-8") as file:
              line = file.readline()
              while line:
                  self.parse_line(line)
                  line = file.readline()

In [7]:
import os.path
from typing import List

import cv2
import librosa
import numpy as np
import pandas as pd
import skimage
from sklearn.preprocessing import LabelBinarizer
from sklearn.preprocessing import MinMaxScaler




In [71]:
def scale_minmax(X, min=0.0, max=1.0):
    X_std = (X - X.min()) / (X.max() - X.min())
    X_scaled = X_std * (max - min) + min
    return X_scaled


def create_images(paths, base_path, max):
    for i, p in enumerate(paths):
      if i >=max:
        break
      temp = f"{base_path}/images/{os.path.basename(os.path.dirname(p[1]))}.png"
      if os.path.exists(temp):
          pass
      else:
          create_image(p[1], base_path)


def create_image(audio_path, base_path):
    # settings
    hop_length = 512  # number of samples per time-step in spectrogram
    n_mels = 128  # number of bins in spectrogram. Height of image
    time_steps = 384  # number of time-steps. Width of image

    # load audio. Using example from librosa

    y, sr = librosa.load(audio_path, sr=44100)
    out_pure_data = f"{base_path}/images/{os.path.basename(os.path.dirname(audio_path))}.png"

    # extract a fixed length window
    start_sample = 0  # starting at beginning
    length_samples = time_steps * hop_length
    # window = y[start_sample:start_sample + length_samples]
    window = y
    # convert to PNG
    spectrogram_image(window, sr=sr, out=out_pure_data, hop_length=hop_length, n_mels=n_mels)
    print('wrote file', out_pure_data)


def spectrogram_image(y, sr, out, hop_length, n_mels):
    # use log-melspectrogram
    mels = librosa.feature.melspectrogram(y=y, sr=sr, n_mels=n_mels,
                                          n_fft=hop_length * 2, hop_length=hop_length)
    mels = np.log(mels + 1e-9)  # add small number to avoid log(0)

    # min-max scale to fit inside 8-bit range
    img = scale_minmax(mels, 0, 255).astype(np.uint8)
    img = np.flip(img, axis=0)  # put low frequencies at the bottom in image
    img = 255 - img  # invert. make black==more energy

    # save as PNG
    skimage.io.imsave(out, img)


def contains_any_index(root, a_list):
    for i, c in enumerate(a_list):
        if c.startswith(root):
            return i + 1
    return 0


def get_paths(dir_path):
    file_paths = []
    audio_paths = []
    for root, directories, files in os.walk(dir_path):
        for filename in files:
            if filename.endswith(".mp3"):
                audio_paths.append(os.path.join(root, filename))
    for root, directories, files in os.walk(dir_path):
        for filename in files:
            if not filename.endswith(".mp3"):
                filepath = os.path.join(root, filename)
                audio_path_index = contains_any_index(root, audio_paths)
                if not audio_path_index == 0:
                    file_paths.append((filepath, audio_paths[audio_path_index-1]))
    # returning all file paths
    return file_paths


def load_spectrogramm_image(paths,max):
  # image = np.zeros((64, 64, 3), dtype="uint8")
  images = []
  for i, path in enumerate(paths):
    if i>= max:
      break
    image = cv2.imread(path)
    images.append(image)
  return np.array(images)

In [100]:
def load_beatmap_attributes(path):
    cols = ["x", "y", "time", "type", "endtime", "x2", "2", "x3", "y3", "x4", "y4", "slide", "length"]

    parser = Parse()
    parser.parse_file(path)
    max_hit_object = 2000
    data = np.zeros((13, max_hit_object), dtype=object)

    for (i, o) in enumerate(parser.hit_objects):

        if i >= max_hit_object:
            break

        else:

            data[0][i] = o.x
            data[1][i] = o.y
            data[2][i] = o.time
            data[3][i] = o.type

            if isinstance(o, Cercle):
                pass
            elif isinstance(o, Spinner):
                data[4][i] = o.endTime
            elif isinstance(o, Slider):
                data[5][i] = o.curvePoints[0].x
                data[6][i] = o.curvePoints[0].y

                if len(o.curvePoints) > 1:
                    data[7][i] = o.curvePoints[1].x
                    data[8][i] = o.curvePoints[1].y

                if len(o.curvePoints) > 2:
                    data[9][i] = o.curvePoints[2].x
                    data[10][i] = o.curvePoints[2].y

                data[11][i] = o.slides
                data[12][i] = o.length

    df_t = pd.DataFrame(data).T
    df = pd.DataFrame(df_t.values.tolist(), columns=cols)
    df = pd.DataFrame(df.values.tolist()).T
    # df.drop(df.tail(1).index,
    #         inplace=True)
    # print(df.loc[len(parser.hit_objects), :])
    return data, parser.difficulty.OverallDifficulty

In [57]:

def load_beatmaps(paths,max):
    arr = []
    diff = []
    for i,path in enumerate(paths):
      if i>=max:
        break
      df_temp, difficulty = load_beatmap_attributes(path[0])
      # print(df_temp.shape)
      arr.append(df_temp)
      diff.append(difficulty)
    # print(np.array(arr, dtype=object).shape)
    diff = np.array(diff, dtype=float)
    df = np.asarray(arr, dtype=object)
    return df, diff

In [30]:
# import the necessary packages
from keras.engine.input_layer import InputLayer
from keras.models import Sequential
from keras.layers import BatchNormalization
from keras.layers import Conv2D
from keras.layers import MaxPooling2D
from keras.layers import Activation
from keras.layers import Dropout
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers import Input
from keras.models import Model
from keras.layers import concatenate

In [31]:
def create_mlp(dim, regress=False):
    # define our MLP network
    model = Sequential()
    model.add(Dense(32, input_dim=dim, activation="relu"))
    model.add(Dense(32, activation="relu"))
    # check to see if the regression node should be added
    if regress:
        model.add(Dense(32, activation="linear"))
    # return our model
    return model

In [92]:
def create_difficulty(shape):
  extra = Sequential()
  extra.add(Activation('sigmoid', input_shape=(shape,)))
  print(extra.output_shape)
  return extra

In [33]:
def create_cnn(width, height, depth, filters=(16, 32, 64, 128, 256), regress=False):
    # initialize the input shape and channel dimension, assuming
    # TensorFlow/channels-last ordering
    inputShape = (height, width, depth)
    chanDim = -1
    # define the model input
    inputs = Input(shape=inputShape)
    # loop over the number of filters
    for (i, f) in enumerate(filters):
        # if this is the first CONV layer then set the input
        # appropriately
        if i == 0:
            x = inputs
        # CONV => RELU => BN => POOL
        x = Conv2D(f, (3, 3), padding="same")(x)
        x = Activation("relu")(x)
        x = BatchNormalization(axis=chanDim)(x)
        x = MaxPooling2D(pool_size=(2, 2))(x)

        # flatten the volume, then FC => RELU => BN => DROPOUT
        x = Flatten()(x)
        x = Dense(64)(x)
        x = Activation("relu")(x)
        x = BatchNormalization(axis=chanDim)(x)
        x = Dropout(0.5)(x)
        # apply another FC layer, this one to match the number of nodes
        # coming out of the MLP
        x = Dense(32)(x)
        x = Activation("relu")(x)
        # check to see if the regression node should be added
        if regress:
            x = Dense(16, activation="linear")(x)
        # construct the CNN
        model = Model(inputs, x)
        # return the CNN
        return model

In [34]:


def get_model(trainAttrX_shape, trainDiffX_shape):
    # create the MLP and CNN models
    mlp = create_mlp(trainAttrX_shape, regress=False)
    cnn = create_cnn(64, 64, 3, regress=False)
    diff = create_difficulty(trainDiffX_shape)
    # create the input to our final set of layers as the *output* of both
    # the MLP and CNN
    combinedInput = concatenate([mlp.output, cnn.output, diff.output])

    x = Dense(64, activation="relu")(combinedInput)
    x = Dense(32, activation="linear")(x)
    # our final model will accept music spectrogram image/difficulty data on the MLP
    # input and images on the CNN input, outputting an array of the same dim as the input of the MLP
    model = Model(inputs=[cnn.input, diff.input], outputs=mlp.input)
    return model

In [98]:
# import the necessary packages

from sklearn.model_selection import train_test_split
import tensorflow as tf
from keras.layers import Dense
from keras.models import Model
from keras.optimizers import Adam
from keras.layers import concatenate
import numpy as np
# import argparse
import locale
import os




In [13]:
# construct the argument parser and parse the arguments
# ap = argparse.ArgumentParser()
# ap.add_argument("-d", "--dataset", type=str, required=True,
# 	help="path to input dataset of house images")
# args = vars(ap.parse_args())

# construct the path to the input .txt file that contains information
# on each house in the dataset and then load the dataset
print("[INFO] loading beatmap attributes...")
base_path = "/content/drive/MyDrive"

paths = get_paths(base_path)


[INFO] loading beatmap attributes...


In [64]:
MAX_MAP = 5

In [65]:
df, difficulty = load_beatmaps(paths,MAX_MAP)

In [63]:
create_images(paths, "/content/datasets", MAX_MAP)

In [73]:
# load the spectrogram images and then scale the pixel intensities to the
# range [0, 1]
print("[INFO] loading spectrogram images...")
images = load_spectrogramm_image("/content/datasets/images",MAX_MAP)


[INFO] loading spectrogram images...


In [74]:
# partition the data into training and testing splits using 75% of
# the data for training and the remaining 25% for testing
print("[INFO] processing data...")
split = train_test_split(df, images, difficulty, test_size=0.25, random_state=42)
(trainAttrX, testAttrX, trainImagesX, testImagesX, trainDifficultyX, testDifficultyX) = split

[INFO] processing data...


In [78]:
trainAttrX_shape = trainAttrX.shape[1]

In [85]:
trainDiif_shape= 1

In [93]:
# create the MLP and CNN models
mlp = create_mlp(trainAttrX_shape, regress=False)
cnn = create_cnn(64, 64, 3, regress=False)
diff = create_difficulty(trainDiif_shape)
# create the input to our final set of layers as the *output* of both
# the MLP and CNN
combinedInput = concatenate([mlp.output, cnn.output, diff.output])

x = Dense(64, activation="relu")(combinedInput)
x = Dense(32, activation="linear")(x)
# our final model will accept music spectrogram image/difficulty data on the MLP
# input and images on the CNN input, outputting an array of the same dim as the input of the MLP
model = Model(inputs=[cnn.input, diff.input], outputs=mlp.input)

(None, 1)


In [94]:
model = get_model(trainAttrX_shape,trainDiif_shape)

(None, 1)


In [95]:
# compile the model using mean absolute percentage error as our loss,
# implying that we seek to minimize the absolute percentage difference
# between our price *predictions* and the *actual prices*
opt = Adam(lr=1e-3, decay=1e-3 / 200)
model.compile(loss="mean_absolute_percentage_error", optimizer=opt)

/usr/local/lib/python3.10/dist-packages/keras/optimizers/legacy/adam.py:117: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)


In [108]:
trainAttrX = tf.convert_to_tensor(
    trainAttrX, dtype="float32"
)
trainImagesX = tf.convert_to_tensor(
    trainImagesX
)
trainDifficultyX = tf.convert_to_tensor(
    trainDifficultyX
)

In [107]:
testAttrX = tf.convert_to_tensor(
    testAttrX, dtype="float32"
)
testImagesX = tf.convert_to_tensor(
    testImagesX
)
testDifficultyX = tf.convert_to_tensor(
    testDifficultyX
)

ValueError: ignored

In [96]:
# train the model
print("[INFO] training model...")
model.fit(
	x=[trainAttrX, trainImagesX,trainDifficultyX], y=trainAttrX,
	validation_data=([testAttrX, testImagesX, testDifficultyX], testAttrX),
	epochs=200, batch_size=8)


[INFO] training model...


ValueError: ignored

In [ ]:
# make predictions on the testing data
print("[INFO] predicting house prices...")
preds = model.predict([testImagesX,testDifficultyX])